Element pattern
====
Last updated: 5 Oct 2018  

This notebook will read a CST antenna pattern file and save in a pickle format data file for use by other notebooks.  


This notebook will also calculate the pattern for a circular disk element using equations from reference [1].  

From Working Group 2 meeting, the element has a directivity of 5.6 dBi at L-Band.  
| Frequency (MHz) | 1435 | 1480 | 1525 |
|:----------------|:----:|:----:|:----:|
| N elements | 140 | 140 | 140 |
| Element directivity (dBi) | 5.6 | 5.6 | 5.6 |


Reference:  
1) Microstrip Antenna Design Handbook, by Ramesh Garg, Prakash Bhartia, Inder Bahl, and Apisak Ittipihoon, Artech House,2001, ISBN 0-89006-513-6.  See page 324 for radiation fields of a circular patch.  

In [1]:
import os
import pickle
import numpy as np
from scipy import special
import matplotlib.pyplot as plt

# Read CST antenna pattern file
For now this is a junk file exported from CST.  The file is from a conformal array of elements.  
Need to pay attenstion to the orientation of theta and phi.  
header from data file:  
```
// CST Farfield Source File
// Version: 3.0 
// Data Type Farfield 
// #Frequencies 1 
// Position 2.255973e-017 0.000000e+000 -8.354133e-002 
// zAxis 0.000000e+000 1.000000e+000 0.000000e+000 
// xAxis 0.000000e+000 0.000000e+000 1.000000e+000 
// Radiated/Accepted/Stimulated Power , Frequency 1.905259e+000 1.981235e+000 2.000000e+000 2.240000e+009 
// >> Total #phi samples, total #theta samples 361 181
// >> Phi, Theta, Re(E_Theta), Im(E_Theta), Re(E_Phi), Im(E_Phi): 
```

In [0]:
# read the pattern data from the CTS file
A = np.loadtxt('Antenn Pattern from CST.ffs')
print('length of data set = {:d} lines'.format(len(A)))

In [0]:
# create arrays to hold the pattern data
step = 1
theta_min = 0
theta_max = 180
phi_min = 0
phi_max = 360
theta = np.arange(theta_min,theta_max+1,step) # include end points
phi = np.arange(phi_min,phi_max+1,step)

E_theta = np.empty((len(theta),len(phi)), dtype=complex)
E_phi = np.empty((len(theta),len(phi)), dtype=complex)

# digest the data
for k in range(len(A)):
    i = int(A[k,1])  # theta index
    j = int(A[k,0])  # phi index
    E_theta[i,j] = A[k,2] + A[k,3]*1J
    E_phi[i,j] = A[k,4] + A[k,5]*1J

In [0]:
# export element pattern
R = np.sqrt(E_theta**2+E_phi**2)
pickle.dump(R, open( "EP.pkl", "wb" ))

In [0]:
# Plot the pattern
F = np.sqrt(E_theta**2+E_phi**2)
F_plt = 20*np.log10(abs(F)) # convert to dB magnitude for plotting
F_plt  = np.clip(F_plt,-40,np.inf) # set limits for plotted data

# re-build theta and phi in terms of deg
theta = np.arange(theta_min,theta_max+1,step) # include end points
phi = np.arange(phi_min,phi_max+1,step)
X1, Y1 = np.meshgrid(phi, theta)

plt.contourf(X1,Y1,F_plt,10)
plt.grid(True)
plt.xlabel('phi')
plt.ylabel('theta')
plt.ylim(theta_min,theta_max)
plt.yticks(np.arange(theta_min, theta_max+1, step=45))
plt.xlim(phi_min,phi_max)
plt.xticks(np.arange(phi_min, phi_max+1, step=45))
plt.colorbar(format='$%.0f$')
plt.show()

In [0]:
# plotting principle plane cuts
plt.plot(theta,F_plt[:,0],'b',label='phi=0')
plt.plot(theta,F_plt[:,45],'r',label='phi=45')
plt.grid(True)
plt.legend()
plt.xlabel('theta')
plt.ylabel('amplitude, dB')
plt.ylim(-10,30)
plt.xlim(theta_min,theta_max)
plt.xticks(np.arange(theta_min, theta_max+1, step=45))
plt.show()

In [0]:
# plot data in polar format
# The 111 specifies 1 row, 1 column on subplot #1
ax = plt.subplot(111, polar=True)
ax.grid(True)
minGrid = -10 #set plot grid
maxGrid = 30
gridSpacing = 5
ax.set_yticks(np.arange(minGrid,maxGrid,gridSpacing))
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
ax.autoscale(enable=False)

# Polar plot
ax.plot(theta*np.pi/180,F_plt[:,0],'b',linewidth = 2,label='phi=0')
ax.plot(theta*np.pi/180,F_plt[:,45],'r',linewidth = 2,label='phi=45')

# Setting the graph title & legend
ax.set_title("test title")
plt.legend()
plt.show()

# Model radiation pattern for a circular disk
Equations 5.19 and 5.20 from reference [1] are used to model a circular disk antenna.  Section 5.2.1 describes a cavity model for the circular element.  Radiation fields are developed from the model.  Assuming a mode corresponding to n = m = 1, the minimum radius and resonance frequency is calculated.  

The far field values obtained from equations 5.19 and 5.20 are adquate for the initial analysis.  Actual radiation patterns will be used when they are available.

See equation 5.17 for V  

$V=hE_nJ_n(X_{nm})$  

V is known as the edge voltage at phi = 0.  
En = electric field for n  
Jn is the Bessel function of order n.  

The far fields are delineated by the condition that:  
$r >> r'$  or $r \geq \frac {2L^2}{\lambda _0}$  
where L is the largest dimension of the aperature.  

In [2]:
# find wave length for the L-Band element
f = 1435 # MHz
c = 299792458 # speed of light m/s
L = c/(f*1e6) # wave length

# paramters used in the model
a = (L/4)/2 # disk metalization radius 1/4 Lambda dia patch
ko = 1 # free space wave number
eo = 8.85e-12 # free space dielectric constant
er = 1 # air
h = 0.00635 # 0.25 in
V = 1 # edge voltage
LL = 0.0254  # 4 in
r = 1 # distance to observation point, m
n = 1 # order for Bessel function

The E-plane radiation fields should be multiplied by $F_3(\theta)$ and the H-plane data fields should by $F_4(\theta)$ per disscussion in section 4.3 of reference [1].  The above expressions for radiation fields assume that the radiating slots are in free spce.  The effects of theground plane and substraite of the antenna on the radiation pattern can be included by means of factors $F_3(\theta)$ and $F_4(\theta)$, equations 4.41 and 4.42 in reference [1].  Also see 2.59 in chapter two of reference [1].  

Need to model the CBITS element and adjust paramters above.  

Calculate and plot for $\theta$ = $-\pi$ to $\pi$

In [3]:
step = 1
theta_min = -90
theta_max = 90
phi_min = 0
phi_max = 360

theta = np.arange(theta_min,theta_max+1,step,dtype=float) # include end points
theta[np.where(theta == 0)[0]] = 0.001  # remove zero value, prevents divide by zero
theta[0] = theta_min + 0.1 # using -89.9 for -90 prevents divide by zero
theta[-1] = theta_max - 0.1
theta = theta*np.pi/180 # convert to radians
phi = np.arange(phi_min,phi_max,step,dtype=float)*np.pi/180

E_theta = np.zeros((len(theta),len(phi)),dtype=complex)
E_phi = np.zeros((len(theta),len(phi)),dtype=complex)

for i in range(len(theta)):
    for j in range(len(phi)):
        F3n = 2*np.cos(theta[i])*np.sqrt(er-np.sin(theta[i])**2) # F3 numerator
        F3d = np.sqrt(er-np.sin(theta[i])**2) - 1j*er*np.cos(theta[i])*np.arctan(ko*h*np.sqrt(er-np.sin(theta[i])**2)) # F3 denominator
        F3 = (F3n/F3d) # eq 4.41
        E_theta [i,j] = ((-1j**n)*V*a*ko/2)*(np.exp(-1j*ko*r)/r)*np.cos(n*phi[j])*special.jvp(1,ko*a*np.sin(theta[i]))*F3 # eq 5.19

        F4n = 2*np.cos(theta[i]) # F4 numberator
        F4d = np.cos(theta[i]) - 1j*np.sqrt(er-np.sin(theta[i])**2)*np.arctan(ko*h*np.sqrt(er-np.sin(theta[i])**2)) # F4 denominator
        F4 = (F4n/F4d) # eq 4.42
        p1 = (n*(1j**n))*(V*a*ko/2)*(np.exp(-1j*ko*r)/r)
        p2 = np.sin(n*phi[j])*(special.jv(1,ko*a*np.sin(theta[i]))/(ko*a*np.sin(theta[i])))*np.cos(theta[i])
        E_phi[i,j] = p1*p2*F4 # eq 5.20

In [2]:
120*np.pi

376.99111843077515

In [4]:
# convert E to intensity, E to U
Zo = 120*np.pi
U = (1/Zo)*np.sqrt(E_theta**2+E_phi**2)**2 # I think the variable should be E(theta,phi) = electric field w/ units of V/m

In [6]:
# normalize the intensity pattern
E = np.sqrt(E_theta**2+E_phi**2) # I think the variable should be E(theta,phi) = electric field w/ units of V/m

## save to data file

In [0]:
# export element pattern
pickle.dump(E, open( "EP.pkl", "wb" ))

## Plot element electric field

In [0]:
# plot the data
E_plt = 20*np.log10(abs(E)) # convert to dB magnitude for plotting
E_plt  = np.clip(U_plt,-60,np.inf) # set limits for plotted data

# re-build theta and phi in terms of deg
theta = np.arange(theta_min,theta_max+1,step) # include end points
phi = np.arange(phi_min,phi_max,step)
X1, Y1 = np.meshgrid(phi, theta)

plt.contourf(X1,Y1,E_plt,10)
plt.grid(True)
plt.xlabel('phi')
plt.ylabel('theta')
plt.ylim(theta_min,theta_max)
plt.yticks(np.arange(theta_min, theta_max+1, step=45))
plt.xlim(phi_min,phi_max)
plt.xticks(np.arange(phi_min, phi_max+1, step=45))
plt.colorbar(format='$%.0f$')
plt.title('electric field')
plt.show()

In [0]:
# plotting principle plane cuts
plt.plot(theta,E_plt[:,0],'b',label='phi=0')
plt.plot(theta,E_plt[:,int(90/step)],'r',label='phi=90')
plt.grid(True)
plt.legend()
plt.xlabel('theta')
plt.ylabel('amplitude, dB')
plt.ylim(-40,0)
plt.xlim(theta_min,theta_max)
plt.xticks(np.arange(theta_min, theta_max+1, step=45))
plt.title('electric field')
plt.show()

In [0]:
# plot data in polar format
theta_min = -90
theta_max = 90
theta = np.arange(theta_min,theta_max+1,step) # include end points
# The 111 specifies 1 row, 1 column on subplot #1
ax = plt.subplot(111, polar=True)
ax.grid(True)
minGrid = -40 #set plot grid
maxGrid = 0
gridSpacing = 5
ax.set_yticks(np.arange(minGrid,maxGrid,gridSpacing))
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
ax.autoscale(enable=False)

# Polar plot
ax.plot(theta*np.pi/180,E_plt[:,0],'b',linewidth = 2,label='phi=0')
ax.plot(theta*np.pi/180,E_plt[:,45],'r',linewidth = 2,label='phi=90')

# Setting the graph title & legend
ax.set_title("electric field")
plt.legend()
plt.show()

# Directivity
See page 101 of Antenna book.  

$D=\frac {4\pi}{\int _{0}^{{2\pi }}\int _{0}^{\pi} \vert \frac {E(\theta,\phi)}{E_{max}} \vert^2 \sin(\theta )\,d\theta \,d\phi}$

In [7]:
E_max = np.abs(E).max()

# limits of integration
theta_min = 0
theta_max = 180
phi_min = 0
phi_max = 360
theta = np.arange(theta_min,theta_max+1,step)*np.pi/180 # include end points
phi = np.arange(phi_min,phi_max,step)*np.pi/180

# multiply the antenna intensity pattern by sine(theta)
# integrate over every column
I = np.zeros(len(phi))
for i in range(len(phi)):
    I[i] = np.trapz((np.abs(E[:,i]/E_max)**2)*np.sin(theta),theta)

# then an integral over the result
D = 4*np.pi/np.trapz(I,phi)

print('directivity = {:f} dB'.format(10*np.log10(D)))

directivity = 1.773254 dB


# Directivity
See page 101 of Antenna book.  

$D=\frac {4\pi}{\int _{0}^{{2\pi }}\int _{0}^{\pi} \frac {U(\theta,\phi)}{U_{max}} \sin(\theta )\,d\theta \,d\phi}$

In [5]:
U_max = np.abs(U).max()

# limits of integration
theta_min = 0
theta_max = 180
phi_min = 0
phi_max = 360
theta = np.arange(theta_min,theta_max+1,step)*np.pi/180 # include end points
phi = np.arange(phi_min,phi_max,step)*np.pi/180

# multiply the antenna intensity pattern by sine(theta)
# integrate over every column
I = np.zeros(len(phi))
for i in range(len(phi)):
    I[i] = np.trapz(np.abs(U[:,i]/U_max)*np.sin(theta),theta)

# then an integral over the result
D = 4*np.pi/np.trapz(I,phi)

print('directivity = {:f} dB'.format(10*np.log10(D)))

directivity = 1.773254 dB


# Total radiated power
The total radiated power can be found by integrating over all directions:  

$P_{rad}=\int _{0}^{{2\pi }}\int _{0}^{\pi}U(\theta ,\phi )\sin(\theta )\,d\theta \,d\phi$

In [0]:
# limits of integration
theta_min = 0
theta_max = 180
phi_min = 0
phi_max = 360
theta = np.arange(theta_min,theta_max+1,step)*np.pi/180 # include end points
phi = np.arange(phi_min,phi_max,step)*np.pi/180

# multiply the antenna intensity pattern by sine(theta)
# integrate over every column
I = np.zeros(len(phi))
for i in range(len(phi)):
    I[i] = np.trapz(np.abs(U[:,i])*np.sin(theta),theta)

# then an integral over the result
P_rad = np.trapz(I,phi)

print('radiated power = {:f} dB'.format(10*np.log10(P_rad)))

# Directivity
$D=4\pi \left({\frac {U_{{{\mathrm  {max}}}}}{P_{{{\mathrm  {rad}}}}}}\right)$

In [0]:
U_max = np.abs(U.max())
print('U_max = {:.1f} dB'.format(10*np.log10(U_max)))
D = 4*np.pi*(U_max/P_rad)
print('directivity = {:f} dB'.format(10*np.log10(D)))

# Gain
Since the antenna is specified as being lossless the radiation efficiency is 1. The maximum gain is then equal to:  

$G=A_{eff}\,D$  

$G_{dBi}=10\log _{{10}}(G)$  

In [0]:
A_eff = 1  # antenna efficiency
G = A_eff*D
print('G = {:f}'.format(G))
print('G = {:.2f} dBi'.format(10*np.log10(G)))

# OLD CODE